In [62]:
pip install dash dash-bootstrap-components


Note: you may need to restart the kernel to use updated packages.


In [63]:
#Using Pandas as the data processing package.
import pandas as pd
from datetime import datetime

#Using plotly as the visualization package.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, dcc, html, Input, Output, callback

#for formatting, we will import the dash bootstrap components.
import dash_bootstrap_components as dbc;



In [64]:
UnemployedData = pd.read_csv("Unemployment.csv")
UnemployedData['Category'] = UnemployedData['Category'].replace('Male,Asiain', 'Male,Asian')

Unempoy_DataCA = pd.read_csv("CAUR.csv")
Unempoy_DataCA['DATE'] = pd.to_datetime(Unempoy_DataCA['DATE'])
Unempoy_DataCA['Year'] = Unempoy_DataCA['DATE'].dt.year
Unempoy_DataCA['State'] = 'CA'
Unempoy_DataCA = Unempoy_DataCA.rename(columns={"CAUR":"Rate "})
Unempoy_DataND = pd.read_csv("NDUR.csv")
Unempoy_DataND['DATE'] = pd.to_datetime(Unempoy_DataND['DATE'])
Unempoy_DataND['Year'] = Unempoy_DataND['DATE'].dt.year
Unempoy_DataND['State'] = 'ND'
Unempoy_DataND = Unempoy_DataND.rename(columns={"NDUR":"Rate "})
Unempoy_DataNY = pd.read_csv("NYUR.csv")
Unempoy_DataNY['DATE'] = pd.to_datetime(Unempoy_DataNY['DATE'])
Unempoy_DataNY['Year'] = Unempoy_DataNY['DATE'].dt.year
Unempoy_DataNY['State'] = "NY"
Unempoy_DataNY = Unempoy_DataNY.rename(columns={"NYUR":"Rate "})
Unempoy_Data = pd.concat([Unempoy_DataCA, Unempoy_DataND, Unempoy_DataNY], ignore_index=True)

USEconomyData = pd.read_csv("GDPA.csv")
USEconomyData['DATE'] = pd.to_datetime(USEconomyData['DATE'])
USEconomyData['Year'] = USEconomyData['DATE'].dt.year

RentalvacancyDataCA = pd.read_csv("CARVAC.csv")
RentalvacancyDataCA['DATE'] = pd.to_datetime(RentalvacancyDataCA['DATE'])


# Extract the year from the date column and create a new column 'Year'
RentalvacancyDataCA['Year'] = RentalvacancyDataCA['DATE'].dt.year
RentalvacancyDataCA['State'] = 'CA'
RentalvacancyDataCA = RentalvacancyDataCA.rename(columns={"CARVAC":"RentalVacancy"})

RentalvacancyDataND = pd.read_csv("NDRVAC.csv")
RentalvacancyDataND['DATE'] = pd.to_datetime(RentalvacancyDataND['DATE'])

# Extract the year from the date column and create a new column 'Year'
RentalvacancyDataND['Year'] = RentalvacancyDataND['DATE'].dt.year
RentalvacancyDataND['State'] = 'ND'
RentalvacancyDataND = RentalvacancyDataND.rename(columns={"NDRVAC":"RentalVacancy"})

RentalvacancyDataNY = pd.read_csv("NYRVAC.csv")
RentalvacancyDataNY['DATE'] = pd.to_datetime(RentalvacancyDataNY['DATE'])

# Extract the year from the date column and create a new column 'Year'
RentalvacancyDataNY['Year'] = RentalvacancyDataNY['DATE'].dt.year
RentalvacancyDataNY['State'] = 'NY'
RentalvacancyDataNY = RentalvacancyDataNY.rename(columns={"NYRVAC":"RentalVacancy"})

RentalvacancyData = pd.concat([RentalvacancyDataCA, RentalvacancyDataND, RentalvacancyDataNY], ignore_index=True)

#House index for CA 
HouseIndexDataCA  = pd.read_csv("CASTHPI.csv")
HouseIndexDataCA['DATE'] = pd.to_datetime(HouseIndexDataCA['DATE'])
# Extract the year from the date column and create a new column 'Year'
HouseIndexDataCA['Year'] = HouseIndexDataCA['DATE'].dt.year
# Add a new column as state 
HouseIndexDataCA['State'] = 'CA'
#Rename CASTHPI Column
HouseIndexDataCA = HouseIndexDataCA.rename(columns={"CASTHPI":"HouseIndexRate"})


HouseIndexDataND  = pd.read_csv("NDSTHPI.csv")
HouseIndexDataND['DATE'] = pd.to_datetime(HouseIndexDataND['DATE'])
# Extract the year from the date column and create a new column 'Year'
HouseIndexDataND['Year'] = HouseIndexDataND['DATE'].dt.year
# Add a new column as state 
HouseIndexDataND['State'] = 'ND'
#Rename CASTHPI Column
HouseIndexDataND = HouseIndexDataND.rename(columns={"NDSTHPI":"HouseIndexRate"})


HouseIndexDataNY  = pd.read_csv("NYSTHPI.csv")
HouseIndexDataNY['DATE'] = pd.to_datetime(HouseIndexDataNY['DATE'])
# Extract the year from the date column and create a new column 'Year'
HouseIndexDataNY['Year'] = HouseIndexDataNY['DATE'].dt.year
# Add a new column as state 
HouseIndexDataNY['State'] = 'NY'
#Rename CASTHPI Column
HouseIndexDataNY = HouseIndexDataNY.rename(columns={"NYSTHPI":"HouseIndexRate"})

HouseIndexData = pd.concat([HouseIndexDataCA, HouseIndexDataND, HouseIndexDataNY], ignore_index=True)

In [65]:
desired_years = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2018, 2019, 2020, 2021, 2022, 2023]

desired_years_trend =[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,2013,2014,2015,2016,2017, 2018, 2019, 2020, 2021, 2022, 2023]

desired_years_Recession = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]

desired_years_Covid = [2018, 2019, 2020, 2021, 2022, 2023]

In [66]:
def TimePeriod(Year):
    
    period = ""    
    
    if (Year <=(2007)):  
        period = "Pre-Recession"
    elif((2008)<= Year <=(2009)):
        period = "Recession"
    elif((2010)<= Year <=(2012)):
        period = "Post-Recession"
    elif((2013)<= Year <=(2019)):
        period = "Pre-Covid"
    elif((2020)<= Year <=(2021)):
        period = "Covid"
    elif((2022)<= Year <=(2023)):
        period = "Post-Covid"

    return period
   
# I then add a column called Period to my data 
# and calculate the value of the column using the value of the date column 
# and calling the TimePeriod Function for every value in Year (called theYear)
UnemployedData['Period'] = UnemployedData["Year"].apply(lambda theYear: TimePeriod(theYear))
RentalvacancyData["Period"] = RentalvacancyData["Year"].apply(lambda theYear: TimePeriod(theYear));
HouseIndexData["Period"] = HouseIndexData["Year"].apply(lambda theYear: TimePeriod(theYear));


In [67]:
color_discrete_map = {
    "Pre-Recession": "purple",
    "Recession": "pink",          # Red for Recession
    "Post-Recession": "blue",
    "Pre-Covid": "coral",
    "Covid": "green",             # Blue for Covid
    "Post-Covid": "brown"
}

In [68]:

States = sorted(UnemployedData.State.unique());
Sex = sorted(UnemployedData.Sex.unique());
Periods = sorted(UnemployedData.Period.unique());
Categories = sorted(UnemployedData.Category.unique());
Ages = sorted(UnemployedData.Age.unique());

Sta_RV = sorted(RentalvacancyData.State.unique());
#Periods_RV = sorted(RentalvacancyData.Period.unique());

state_HI = sorted(HouseIndexData.State.unique());

In [69]:
app = Dash(
    __name__, suppress_callback_exceptions=True,
    external_stylesheets=[dbc.themes.LUMEN]
)

In [70]:
#Dropdown to select States
SelectState = dcc.Dropdown(
    
    options=States,
    value=States[0],
    clearable=False,
)
#Dropdown to select Sex
SelectSex = dcc.Dropdown(
    
    options=Sex,
    value=Sex[0],
    clearable=False
)

#Dropdown to select category 
SelectCategory = dcc.Dropdown(
    
    options=Categories,
    value= Categories[0],
    clearable=False
)


In [71]:
filtered_data = UnemployedData[UnemployedData['Year'].isin(desired_years)]
# Group by Year and State, and calculate the mean HouseIndexRate
aggregated_data = filtered_data.groupby(['Year', 'State','Period']).agg({'Rate ':'mean'}).reset_index()
# Define the states we want to plot (replace with your actual states)
states_to_plot = ['CA', 'ND', 'NY']
# Define your color mapping here
color_discrete_map = {
    "Pre-Recession": "purple",
    "Recession": "pink",          # Red for Recession
    "Post-Recession": "blue",
    "Pre-Covid": "coral",
    "Covid": "green",             # Blue for Covid
    "Post-Covid": "brown"
} 
# Create a container for your graphs
graphs_container = []
for state in states_to_plot:
    # Filter the data for the state
    state_data = aggregated_data[aggregated_data['State'] == state]
    
    # Create the area chart for the state
    fig = px.bar(
        state_data,
        x="Year",
        y="Rate ",
        color="Period",
        title=f"Unemployment rate for {state}",
        labels={"Year": "Year", "Rate ": "Unemployment rate in (%)"},
        
        color_discrete_map=color_discrete_map
    )
    
    # Customize the layout
    fig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=desired_years
        ),
        yaxis_title="Unemployment rate in (%)"
    )
    
    graph_component = dcc.Graph(figure=fig)
    graphs_container.append(graph_component)

# Add these graphs to your layout
UnemploymentTabContent = dbc.Card([
    dbc.CardHeader(html.H2("Unemployment Rate for All State by Period", className="mt-4 px-4 align-center")),
    dbc.Row([dbc.Col(graph) for graph in graphs_container]),
], className="mx-4")



# The list of Tabs is an arrary of Tabs each with its own label and any styling.
StateUnemploymentTabs = [
dbc.Tab(UnemploymentTabContent,
         
        label = "Unemployment Rate for All States", 
        activeLabelClassName="",
    )    
];


In [72]:
#Dropdown to select States
SelectState_Trend = dcc.Dropdown(
    
    options=States,
    value=States[0],
    clearable=False,
)

UnemploymentAllStatesRecessionTrendChart = dcc.Graph()

@callback(
        Output(UnemploymentAllStatesRecessionTrendChart, "figure"), 
        [Input(SelectState_Trend, "value")
       
       ])
def UpdateUnemploymentAllStatesRecessionTrendChart(selectedState):
    filtered_data = UnemployedData[
        (UnemployedData['State'] == selectedState) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years_Recession)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['Rate '].mean().reset_index()

    title = f"Unemployment Trend during Recession : {selectedState}"
  

    UnemploymentFig = px.bar(
            aggregated_data,
            x="Year",
            y="Rate ", 
            color="Period",
            title=title,
            labels=dict(Year = "Year", Rate = "Unemployment Rate (%)"),
            color_discrete_map=color_discrete_map
    )
    
    UnemploymentFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years_Recession  # Ensure desired_years is defined and in scope
    )
)

    return UnemploymentFig

In [73]:
UnemploymentAllStatesCovidTrendChart = dcc.Graph()

@callback(
        Output(UnemploymentAllStatesCovidTrendChart, "figure"), 
        [Input(SelectState_Trend, "value")

       ])
def UpdateUnemploymentAllStatesCovidTrendChart(selectedState):
    filtered_data = UnemployedData[
        (UnemployedData['State'] == selectedState) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years_Covid)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['Rate '].mean().reset_index()

    title = f"Unemployment Trend during Covid: {selectedState}"
  

    UnemploymentFig = px.bar(
            aggregated_data,
            x="Year",
            y="Rate ", 
            color="Period",
            title=title,
            labels=dict(Year = "Year", Rate = "Unemployment Rate (%)"),
            color_discrete_map=color_discrete_map
    )
    
    UnemploymentFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years_Covid  # Ensure desired_years is defined and in scope
    )
)
    

    return UnemploymentFig

In [74]:
UnemploymentTrendTabContent = dbc.Card([
    dbc.CardHeader([
        html.Div([
            html.H2("Unemployment Trend During Recession and Covid", className="mt-4 px-4 align-center"), 
        ]),
    ]),  # Closes CardHeader

    dbc.CardBody([
        dbc.Row([
            dbc.Col([
                html.Div([
                    dbc.Label("Select a State", className="pl-4"),
                    SelectState_Trend,
                ], className="col-4"),
            ], className="my-4"),

        ]),  
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Unemployment Trend during Recession ", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                UnemploymentAllStatesRecessionTrendChart
            ]),  # Closes Col
        ]),  # Closes the Row for "Unemployment in State"
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Unemployment Trend during Covid", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                UnemploymentAllStatesCovidTrendChart
            ]),  # Closes Col
        ]),  # Closes the Row for "Unemployment in State"
    ], className="mx-4"),  # Closes CardBody
])  # Closes Card

UnemploymentTrendTab = dbc.Tab(
    UnemploymentTrendTabContent,
    label="Unemployment Trend ",
    activeLabelClassName="bg-light",
    
)

StateUnemploymentTabs.append(UnemploymentTrendTab)


In [75]:
State_SelRV_Trend = dcc.Dropdown(
    
    options=Sta_RV,
    value=Sta_RV[0],
    clearable=False
)

In [76]:
RentalvacancyAllStateRecessionTrendChart = dcc.Graph()

@callback(
    Output(RentalvacancyAllStateRecessionTrendChart, "figure"),
    Input(State_SelRV_Trend, "value")
)

def UpdateRentalvacancyAllStateRecessionTrendChart(selectedstate):
    filtered_data = RentalvacancyData[
        (RentalvacancyData['State'] == selectedstate) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years_Recession)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['RentalVacancy'].mean().reset_index()

    title = f"Rental Vacancy Trend during Recession : {selectedstate}"
    
    RentalVacancyFig = px.bar(
        aggregated_data,
        x="Year",
        y="RentalVacancy",  # Make sure the column name matches exactly, including any spaces
        color="Period",
        title=title,
        labels={"Year": "Year", "RentalVacancy": "Rental Vacancy  Rate (%)"},
       
       color_discrete_map=color_discrete_map
    )
    
    RentalVacancyFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years_Recession  # Ensure desired_years is defined and in scope
    ))

    #for trace in RentalVacancyFig.data:
     #   trace.update(stackgroup=None)
    return RentalVacancyFig

In [77]:
RentalvacancyAllStateCovidTrendChart = dcc.Graph()

@callback(
    Output(RentalvacancyAllStateCovidTrendChart, "figure"),
    Input(State_SelRV_Trend, "value")
)

def UpdateRentalvacancyAllStateCovidTrendChartt(selectedstate):
    filtered_data = RentalvacancyData[
        (RentalvacancyData['State'] == selectedstate) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years_Covid)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['RentalVacancy'].mean().reset_index()

    title = f"Rental Vacancy Trend during Covid : {selectedstate}"
    
    RentalVacancyFig = px.bar(
        aggregated_data,
        x="Year",
        y="RentalVacancy",  # Make sure the column name matches exactly, including any spaces
        color="Period",
        title=title,
        labels={"Year": "Year", "RentalVacancy": "Rental Vacancy  Rate (%)"},
        
        color_discrete_map=color_discrete_map
        
    )
    
    RentalVacancyFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years_Covid # Ensure desired_years is defined and in scope
    ))

    
    
    return RentalVacancyFig

In [78]:
RentalVacancyTrendTabContent = dbc.Card([
    dbc.CardHeader([
        html.Div([
            html.H2("Rental Vacancy Trend During Recession and Covid", className="mt-4 px-4 align-center"), 
        ]),
    ]),  # Closes CardHeader

    dbc.CardBody([
        dbc.Row([
            dbc.Col([
                html.Div([
                    dbc.Label("Select a State", className="pl-4"),
                    State_SelRV_Trend,
                ], className="col-4"),
            ], className="my-4"),

        ]),  
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Rental Vacancy Trend during Recession ", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                RentalvacancyAllStateRecessionTrendChart
            ]),  # Closes Col
        ]),  # Closes the Row for "Unemployment in State"
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Rental Vacancy Trend during Covid", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                RentalvacancyAllStateCovidTrendChart
            ]),  # Closes Col
        ]),  # Closes the Row for "Unemployment in State"
    ], className="mx-4"),  # Closes CardBody
])  # Closes Card

RentalVacancyTrendTab = dbc.Tab(
    RentalVacancyTrendTabContent,
    label="Rental Vacancy Trend ",
    activeLabelClassName="bg-light",
    
)

StateUnemploymentTabs.append(RentalVacancyTrendTab)


In [79]:
#Dropdown to select States
Sta_selRV = dcc.Dropdown(
    
    options=Sta_RV,
    value=Sta_RV[0],
    clearable=False
)



In [80]:
# Pre-filter the data to only include the desired years
filtered_data = RentalvacancyData[RentalvacancyData['Year'].isin(desired_years)]

# Group by Year and State, and calculate the mean HouseIndexRate
aggregated_data = filtered_data.groupby(['Year', 'State','Period']).agg({'RentalVacancy':'mean'}).reset_index()

# Define the states we want to plot (replace with your actual states)
states_to_plot = ['CA', 'ND', 'NY']

# Define your color mapping here
color_discrete_map = {
    "Pre-Recession": "purple",
    "Recession": "pink",          # Red for Recession
    "Post-Recession": "blue",
    "Pre-Covid": "coral",
    "Covid": "green",             # Blue for Covid
    "Post-Covid": "brown"
} 

# Create a container for your graphs
graphs_container = []

for state in states_to_plot:
    # Filter the data for the state
    state_data = aggregated_data[aggregated_data['State'] == state]
    
    # Create the area chart for the state
    fig = px.bar(
        state_data,
        x="Year",
        y="RentalVacancy",
        color="Period",
        title=f"Rental Vacancy Index for {state}",
        labels={"Year": "Year", "RentalVacancy": "Rental Vacancy rate in (%)"},
        
        color_discrete_map=color_discrete_map
    )
    
    # Customize the layout
    fig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=desired_years
        ),
        yaxis_title="Rental Vacancy rate in (%)"
    )
    
    graph_component = dcc.Graph(figure=fig)
    graphs_container.append(graph_component)

# Add these graphs to your layout
RentalVacancyTabContent = dbc.Card([
    dbc.CardHeader(html.H2("Rental Vacancy Rate for All State by Period", className="mt-4 px-4 align-center")),
    dbc.Row([dbc.Col(graph) for graph in graphs_container]),
], className="mx-4")

RentalVacancyTab = dbc.Tab(
    RentalVacancyTabContent,
    label="Rental Vacancy(All States)",
    activeLabelClassName="bg-light",
    
)

# Assuming StateUnemploymentTabs is a list or a container for tabs
StateUnemploymentTabs.append(RentalVacancyTab)

    


In [81]:
color_discrete_map_state = {
    'CA': 'blue',
    'ND': 'green',
    'NY': 'orange'
    # Add all states you need to plot
}

In [82]:
SelectState_HI = dcc.Dropdown(
    
    options=state_HI,
    value=state_HI[0],
    clearable=False,
)

In [83]:
HousingIndexAllStateRecessionTrendChart = dcc.Graph()

@callback(
    Output(HousingIndexAllStateRecessionTrendChart, "figure"),
    Input(SelectState_HI, "value")
)

def UpdateHousingIndexAllStateRecessionTrendChart(selectedstate):
    filtered_data = HouseIndexData[
        (HouseIndexData['State'] == selectedstate) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years_Recession)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['HouseIndexRate'].mean().reset_index()

    title = f"HouseIndex  Trend during Recession : {selectedstate}"
    
    HouseIndexFig = px.line(
        aggregated_data,
        x="Year",
        y="HouseIndexRate",  # Make sure the column name matches exactly, including any spaces
        color="Period",
        title=title,
        labels={"Year": "Year", "HouseIndexRate": "House Index Rate (%)"},
        markers=True,
        symbol="Period",
        line_shape="spline",
        color_discrete_map=color_discrete_map
    )
    
    HouseIndexFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years_Recession  # Ensure desired_years is defined and in scope
    ))

    HouseIndexFig.update_traces(marker=dict(size=12))

    return HouseIndexFig

In [84]:
HousingIndexAllStateCovidTrendChart = dcc.Graph()

@callback(
    Output(HousingIndexAllStateCovidTrendChart, "figure"),
    Input(SelectState_HI, "value")
)

def UpdateHousingIndexAllStateRecessionTrendChart(selectedstate):
    filtered_data = HouseIndexData[
        (HouseIndexData['State'] == selectedstate) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years_Covid)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['HouseIndexRate'].mean().reset_index()

    title = f"HouseIndex Trend during Covid : {selectedstate}"
    
    HouseIndexFig = px.line(
        aggregated_data,
        x="Year",
        y="HouseIndexRate",  # Make sure the column name matches exactly, including any spaces
        color="Period",
        title=title,
        labels={"Year": "Year", "HouseIndexRate": "House Index Rate (%)"},
        markers=True,
        symbol="Period",
        line_shape="spline",
        color_discrete_map=color_discrete_map
    )
    
    HouseIndexFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years_Covid  # Ensure desired_years is defined and in scope
    ))

    HouseIndexFig.update_traces(marker=dict(size=12))

    return HouseIndexFig

In [85]:
HouseIndexTrendTabContent = dbc.Card([
    dbc.CardHeader([
        html.Div([
            html.H2("HouseIndex Trend During Recession and Covid", className="mt-4 px-4 align-center"), 
        ]),
    ]),  # Closes CardHeader

    dbc.CardBody([
        dbc.Row([
            dbc.Col([
                html.Div([
                    dbc.Label("Select a State", className="pl-4"),
                    SelectState_HI,
                ], className="col-4"),
            ], className="my-4"),

        ]),  
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("HouseIndex Trend during Recession ", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                HousingIndexAllStateRecessionTrendChart
            ]),  # Closes Col
        ]),  # Closes the Row for "Unemployment in State"
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("HouseIndex Trend during Covid", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                HousingIndexAllStateCovidTrendChart
            ]),  # Closes Col
        ]),  # Closes the Row for "Unemployment in State"
    ], className="mx-4"),  # Closes CardBody
])  # Closes Card

HouseIndexTrendTab = dbc.Tab(
    HouseIndexTrendTabContent,
    label="House Index Trend ",
    activeLabelClassName="bg-light"
)

StateUnemploymentTabs.append(HouseIndexTrendTab)


In [86]:



# Pre-filter the data to only include the desired years
filtered_data = HouseIndexData[HouseIndexData['Year'].isin(desired_years)]

# Group by Year and State, and calculate the mean HouseIndexRate
aggregated_data = filtered_data.groupby(['Year', 'State','Period']).agg({'HouseIndexRate':'mean'}).reset_index()

# Define the states we want to plot (replace with your actual states)
states_to_plot = ['CA', 'ND', 'NY']

# Define your color mapping here
color_discrete_map = {
    "Pre-Recession": "purple",
    "Recession": "pink",          # Red for Recession
    "Post-Recession": "blue",
    "Pre-Covid": "coral",
    "Covid": "green",             # Blue for Covid
    "Post-Covid": "brown"
} 

# Create a container for your graphs
graphs_container = []

# Create a separate area chart for each state
for state in states_to_plot:
    # Filter the data for the state
    state_data = aggregated_data[aggregated_data['State'] == state]
    
    # Create the area chart for the state
    fig = px.bar(
        state_data,
        x="Year",
        y="HouseIndexRate",
        color="Period",
        title=f"House Price Index for {state}",
        labels={"Year": "Year", "HouseIndexRate": "House Price in Thousands ($)"},
        
        color_discrete_map=color_discrete_map
    )
    
    # Customize the layout
    fig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=desired_years
        ),
        yaxis_title="House Price in Thousands ($)"
    )
    
    # Update the traces to change the color of the area
    
    
    # Add a dcc.Graph component for each figure
    graph_component = dcc.Graph(figure=fig)
    graphs_container.append(graph_component)

# Add these graphs to your layout
HouseIndexTabContent = dbc.Card([
    dbc.CardHeader(html.H2("House Index Rate for All State by Period", className="mt-4 px-4 align-center")),
    dbc.Row([dbc.Col(graph) for graph in graphs_container]),
], className="mx-4")

HouseIndexTab = dbc.Tab(
    HouseIndexTabContent,
    label="House Index(All States)",
    activeLabelClassName="bg-light",
    style={"font-size": 32, "color":"#CD7F32"}
)

# Assuming StateUnemploymentTabs is a list or a container for tabs
StateUnemploymentTabs.append(HouseIndexTab)


In [87]:
filtered_data = HouseIndexData[HouseIndexData['Year'].isin(desired_years_trend)]

# Then, group by Year and State, and calculate the mean HouseIndexRate
aggregated_data = filtered_data.groupby(['Year', 'State']).agg({'HouseIndexRate':'mean'}).reset_index()

# Define the states we want to plot
states_to_plot = aggregated_data['State'].unique()
color_discrete_map = {
    'CA': 'blue',
    'ND': 'blue',
    'NY': 'blue'
    # Add all states you need to plot
}

# Create a separate area chart for each state
graphs_container = []
for state in states_to_plot:
    # Filter the data for the state
    state_data = aggregated_data[aggregated_data['State'] == state]
    
    # Create the area chart for the state
    fig = px.area(
        state_data,
        x="Year",
        y="HouseIndexRate",
        title=f"House Price Index for {state}",
        labels={"Year": "Year", "HouseIndexRate": "House Price in Thousands ($)"},
        line_shape="spline"
    )
    
    # Customize the layout
    fig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=desired_years
        )
    )
    
    
    # Update the traces to change the color of the area
    fig.update_traces(line=dict(color=color_discrete_map[state]), fill='tozeroy')
    fig.update_yaxes(range=[0, 1000])
    
    graph_component = dcc.Graph(figure=fig)
    graphs_container.append(graph_component)
     
LeadersLaggardsTabContent =dbc.Card([
    dbc.CardHeader([
       html.Div([
                    html.H2(
                        "Leaders and Laggards Parameters ", 
                        className="mt-4 px-4 align-center"), 
                ]),
            ]),

    
    dbc.Row([dbc.Col(graph) for graph in graphs_container]),
    ], className="mx-4")

LeadersLaggardsTab =     dbc.Tab(
        LeadersLaggardsTabContent,
        label="Leaders",
        activeLabelClassName="bg-light",
        style={"font-size": 32, "color":"#CD7F32"}
    );

StateUnemploymentTabs.append(LeadersLaggardsTab);




In [88]:
filtered_data = Unempoy_Data[Unempoy_Data['Year'].isin(desired_years_trend)]

# Then, group by Year and State, and calculate the mean HouseIndexRate
aggregated_data = filtered_data.groupby(['Year', 'State']).agg({'Rate ':'mean'}).reset_index()

# Assuming 'GDPA' is the column containing the GDP data
gdp_aggregated = USEconomyData.groupby('Year').agg({'GDPA':'mean'}).reset_index()


# Define the states we want to plot
states_to_plot = aggregated_data['State'].unique()
color_discrete_map = {
    'CA': 'blue',
    'ND': 'blue',
    'NY': 'blue'
    # Add all states you need to plot
}



# Create a separate area chart for each state
graphs_container = []
for state in states_to_plot:
    # Filter the data for the state
    state_data = aggregated_data[aggregated_data['State'] == state]
    
    # Create the area chart for the state
    fig = px.area(
        state_data,
        x="Year",
        y="Rate ",
        title=f"Unemployment Rate for {state}",
        labels={"Year": "Year", "Rate ": "Unemployment Rate (%)"},
        line_shape="spline"
    )
    
    # Customize the layout
    fig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=desired_years
        )
    )
    
    
    
    # Update the traces to change the color of the area
    fig.update_traces(line=dict(color=color_discrete_map[state]), fill='tozeroy')
    fig.update_yaxes(range=[0, 15])
    
    graph_component = dcc.Graph(figure=fig)
    graphs_container.append(graph_component)
    
fig_gdp = px.area(
    gdp_aggregated,
    x="Year",
    y="GDPA",
    title="US Economy Over Time",
    labels={"Year": "Year", "GDPA": "GDP Amount (in Billions)"},
    line_shape="spline"
)

# Customize the layout for the GDP chart (optional)
fig_gdp.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years
    )
)

# Update the traces to change the color of the area (optional)
fig_gdp.update_traces(fill='tozeroy')
fig_gdp.update_yaxes(range=[0, max(gdp_aggregated['GDPA']) * 1.1])  # Adjust Y-axis based on GDP 

gdp_graph_component = dcc.Graph(figure=fig_gdp)
graphs_container.append(gdp_graph_component)


     
LaggardsTabContent =dbc.Card([
    dbc.CardHeader([
       html.Div([
                    html.H2(
                        "Laggards Parameters ", 
                        className="mt-4 px-4 align-center"), 
                ]),
            ]),

    
    dbc.Row([dbc.Col(graph) for graph in graphs_container]),
    ], className="mx-4")

LaggardsTab =     dbc.Tab(
        LaggardsTabContent,
        label="Laggards",
        activeLabelClassName="bg-light"
        
    );

StateUnemploymentTabs.append(LaggardsTab);




In [89]:
SelectState = dcc.Dropdown(
    
    options=States,
    value=States[0],
    clearable=False,
)
#Dropdown to select Sex
SelectSex = dcc.Dropdown(
    
    options=Sex,
    value=Sex[0],
    clearable=False
)

#Dropdown to select category 
SelectCategory = dcc.Dropdown(
    
    options=Categories,
    value= Categories[0],
    clearable=False
)
#Sales In State Chart and Callback
UnemploymentAllStatesChart = dcc.Graph()

#Callback for Sales Chart
@callback(
        Output(UnemploymentAllStatesChart, "figure"), 
        [Input(SelectState, "value"),
        Input(SelectSex, "value") 
       
       ]

)

def updateUnemploymentAllStatesChart(selectedState, selectedSex ):
       
    #Have to query twice to handle both the filter
    filtered_data = UnemployedData[
        (UnemployedData['State'] == selectedState) &
        (UnemployedData['Sex'] == selectedSex) 
    ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['Rate '].mean().reset_index()

    title = f"Unemployment over time in: {selectedState}"
  

    UnemploymentFig = px.bar(
            aggregated_data,
            x="Year",
            y="Rate ", 
            color="Period",
            title=title,
            labels=dict(Year = "Year", Rate = "Unemployment Rate (%)"),
            color_discrete_map=color_discrete_map
    )
    
    UnemploymentFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years  # Ensure desired_years is defined and in scope
    )
)

    return UnemploymentFig

#Sales In State Chart and Callback
UnemploymentInlStatesChart = dcc.Graph()

#Callback for Sales Chart
@callback(
        Output(UnemploymentInlStatesChart, "figure"), 
        [Input(SelectState, "value"),
        Input(SelectSex, "value")
        #Input(SelectCategory, "value")
        ]
)
def updateUnemploymentInlStatesChart(selectedState, selectedSex):
    # Initialize an empty figure or a message to prompt user selection
    if None in [selectedState, selectedSex]:
        return {
            'data': [],
            'layout': {
                'title': 'Please select all filters to view data.'
            }
        }
    
    # Continue with the logic only if all inputs have valid values
    filtered_data = UnemployedData[
        (UnemployedData['State'] == selectedState) &
        (UnemployedData['Sex'] == selectedSex) # &
        #(UnemployedData['Category'] == selectedCategory)
        ]
    filtered_data = filtered_data[filtered_data['Year'].isin(desired_years)]
    
    aggregated_data = filtered_data.groupby(['Year', 'Period'])['Rate '].mean().reset_index()

    title = f"Unemployment over time in: {selectedState} for {selectedSex}"

    
    # Proceed to create the figure with Plotly Express
    UnemploymentFig = px.line(
        aggregated_data,
        x="Year",
        y="Rate ",  # Make sure the column name matches exactly, including any spaces
        color="Period",
        title=title,
        labels={"Year": "Year", "Rate": "Unemployment Rate (%)"},
        markers=True,
        symbol="Period",
        line_shape="spline",
        color_discrete_map=color_discrete_map
    )
    
    UnemploymentFig.update_layout(
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=desired_years  # Ensure desired_years is defined and in scope
    ))

    UnemploymentFig.update_traces(marker=dict(size=12))

    return UnemploymentFig


UnemploymentInAgeChart = dcc.Graph()


#Callback for Sales Chart
@callback(
        Output(UnemploymentInAgeChart, "figure"), 
        Input(SelectState, "value"),
        Input(SelectCategory, "value")
)

def updateUnemploymentInAgeChart(selectedState, selectedCategory):
       
    # Combine both the filters for State and Category
    mask = (UnemployedData['State'] == selectedState) & UnemployedData['Age'].str.startswith(selectedCategory)
    Data = UnemployedData[mask]
    Data = Data[Data['Year'].isin(desired_years)]


    # Aggregate the data by Year and Age to get the mean Rate
    Data_agg = Data.groupby(['Year', 'Age']).agg({'Rate ':'mean'}).reset_index()

    Title = "Unemployment over time in: " + selectedState
  
    # Plot the aggregated data
    UnemploymentFig = px.line(
            Data_agg,
            x="Year",
            y="Rate ", 
            color="Age",
            title=Title,
            labels=dict(Year = "Year", Rate = "Unemployment Rate (%)"),
            markers = True,
            symbol = "Age",
            line_shape = "spline",
            color_discrete_map=color_discrete_map
    )
    
    UnemploymentFig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=desired_years  # Make sure to have this array defined
        )
    )


    UnemploymentFig.update_traces(
        marker=dict(size=12),
    )

    return UnemploymentFig
UnemploymentTabContent = dbc.Card([
    dbc.CardHeader([
        html.Div([
            html.H2("Annual Unemployment by Sex", className="mt-4 px-4 align-center"), 
        ]),
    ]),  # Closes CardHeader

    dbc.CardBody([
        dbc.Row([
            dbc.Col([
                html.Div([
                    dbc.Label("Select a State", className="pl-4"),
                    SelectState,  # Make sure StateSelect is the ID of your dcc.Dropdown
                ], className="col-4"),
            ], className="my-4"),
        
            dbc.Col([
                html.Div([
                    dbc.Label("Select a sex", className="pl-4"),
                    SelectSex,  # Make sure SelectSex is the ID of your dcc.Dropdown
                ], className="col-4"),
            ], className="my-4"),
            
            dbc.Col([
                html.Div([
                    dbc.Label("Select a Category", className="pl-4"),
                    SelectCategory,
                ], className="col-7")
            ], className="my-4"),
        ]),  
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Unemployment All States", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                UnemploymentAllStatesChart
            ]),
        ]),  # Closes the Row for "Unemployment All States"
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Unemployment In State", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                UnemploymentInlStatesChart
            ]),  
        ]),  # Closes the Row for "Unemployment In State"
        
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.H5("Unemployment by Age", className="text-center mt-4 py-2 alert-secondary"), 
                ]),
                UnemploymentInAgeChart  # This is the third chart being added
            ]),
        ]),  # Closes the Row for "Unemployment by Age"
    ], className="mx-4"),# Closes CardBody
]) # Closes Card

UnemploymentTab = dbc.Tab(
    UnemploymentTabContent,
    label="Differential Impacts",
    activeLabelClassName="bg-light")

StateUnemploymentTabs.append(UnemploymentTab)


In [90]:
title = dcc.Markdown("Unemployment, Real Estates and Rental Vacancy Figures for Key States Dashboard", 
                     className="", style={"font-size": 32, "color": "#834333"})

#Remember that markdown can be formatted here, just as in all Jupyter Notebooks.
subtitle = dcc.Markdown("*Use the tabs to navigate across visualizations*", 
                     className="", style={"font-size": 12})

In [91]:
# App Layout
app.layout = dbc.Container(
    [
        #In this container, we add the mardown elements for the title and subtitle first,
        dbc.Row([
            dbc.Col([title], width=12, style={}, className="alert-secondary my-2 py-2 text-center")
        ]),

        dbc.Row([
            dbc.Col([subtitle], width=12, style={"text-align": "right", "margin": "auto"})
        ]),

        #And then the set of Tabs in StateSalesTabs.
       dbc.Tabs(StateUnemploymentTabs)
    ]
);

In [92]:
if __name__ == "__main__":
    app.run(
        debug=True, 
        external_stylesheets=[dbc.themes.LITERA],
        port=8053
    )
